In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt

In [30]:
titanic_train = pd.read_csv('titanic_data/train.csv')
titanic_train.head(10)
# titanic_test_X = pd.read_csv("titanic_data/test.csv")
# titanic_test_X.tail(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [38]:
# sex = {'Sex':{'male': 1, 'female': 0}}
# titanic_train.replace(sex, inplace=True)
titanic_train_X = titanic_train.loc[:, titanic_train.columns != 'Survived']
titanic_train_y = titanic_train['Survived'].reset_index(drop=True)

In [39]:
titanic_train_X = pd.get_dummies(titanic_train_X, columns=['Sex', 'Embarked'], dtype=bool)
# titanic_train_X.Cabin.dropna()

In [33]:
titanic_train.loc[titanic_train['Cabin'].isna(), 'Survived'].value_counts()

0    481
1    206
Name: Survived, dtype: int64

In [34]:
scalable_columns = titanic_train_X.select_dtypes(include=np.number).columns
scaler = StandardScaler()
titanic_scaled = titanic_train_X.copy()
titanic_scaled[scalable_columns]= scaler.fit_transform(titanic_train_X[scalable_columns])
titanic_scaled.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
count,8.910000e+02,8.910000e+02,7.140000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,8.910000e+02,891.000000,8.910000e+02
mean,6.379733e-17,-8.772133e-17,2.388379e-16,4.386066e-17,5.382900e-17,3.987333e-18,3.987333e-17,-1.156327e-16,-1.993666e-17,0.000000,-4.984166e-17
std,1.000562e+00,1.000562e+00,1.000701e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562e+00,1.000562,1.000562e+00
min,-1.730108e+00,-1.566107e+00,-2.016979e+00,-4.745452e-01,-4.736736e-01,-6.484217e-01,-7.376951e-01,-1.355574e+00,-4.820427e-01,-0.307562,-1.614710e+00
25%,-8.650540e-01,-3.693648e-01,-6.595416e-01,-4.745452e-01,-4.736736e-01,-4.891482e-01,-7.376951e-01,-1.355574e+00,-4.820427e-01,-0.307562,-1.614710e+00
50%,0.000000e+00,8.273772e-01,-1.170488e-01,-4.745452e-01,-4.736736e-01,-3.573909e-01,-7.376951e-01,7.376951e-01,-4.820427e-01,-0.307562,6.193064e-01
75%,8.650540e-01,8.273772e-01,5.718310e-01,4.327934e-01,-4.736736e-01,-2.424635e-02,1.355574e+00,7.376951e-01,-4.820427e-01,-0.307562,6.193064e-01
max,1.730108e+00,8.273772e-01,3.465126e+00,6.784163e+00,6.974147e+00,9.667167e+00,1.355574e+00,7.376951e-01,2.074505e+00,3.251373,6.193064e-01


In [11]:
enc = OneHotEncoder(handle_unknown='ignore')
titanic_train_scaled = enc.fit_transform(titanic_scaled)
titanic_test_scaled = enc.fit_transform(titanic_test_X)

array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [13]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score

In [14]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
rfc.fit(titanic_train_scaled, titanic_train_y)

RandomForestClassifier(n_estimators=10, random_state=0)

In [16]:
y_pred = rfc.predict(titanic_test_scaled)

ValueError: X has 1549 features, but RandomForestClassifier is expecting 2971 features as input.